In [32]:
import pd3
study = pd3.Study()
study.load_file("/home/reap2020/notebooks/spaghetti.pb")

True

In [95]:
import importlib
import pd3_plot_colors
importlib.reload(pd3_plot_colors)
from pd3_plot_colors import normalize, plot_study, plot_study3D

x = normalize(1,0,0)
y = normalize(0,1,0)
    
plot_study(study) # Markus's code

# Download paraview if you want (just to see hwat they do)
# Find a way to reuse DFS (probably mamek a new function that returns lines)
# Color by slip system

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[[(-5993.209, -742.9513), (-5524.812, -1031.177)], [(-6829.994, 93.83328), (-6312.965, -423.1952)], [(-7915.101, 1178.941), (-7269.52, 533.3599)], [(-9145.565, 2033.129), (-8384.98, 1648.819)], [(-9944.623, 2533.695), (-9145.565, 2033.129)], [(-10040.02, 2607.925), (-9944.623, 2533.695)], [(-8384.98, 1648.819), (-7915.101, 1178.941)], [(-7269.52, 533.3599), (-6829.994, 93.83328)], [(-6312.965, -423.1952), (-5993.209, -742.95

AttributeError: 'list' object has no attribute 'color'